In [1]:
###########
# IMPORTS #
###########
import os
import sys

import os
import re
from sentence_transformers import SentenceTransformer


import os
from IPython.display import clear_output

import numpy as np

import pickle 

assert os.path.isfile('../config/repository_path.txt')
with open('../config/repository_path.txt', 'r') as reader:
    repository_path = reader.read()
sys.path.append(repository_path)

from lib import dataset
from lib.scal import SCALRefugees
from lib.scal_ui import SCAL_UI
##########
# PARAMS #
##########
seed = 2474 #int(np.random.default_rng(2022).random()*10000)
cap=4
N=50000
second_round=False
#---

# ----- #
# INPUT #
# ----- #
model_path='/home/ec2-user/SageMaker/mariano/sentence_bert/pretrained/all-MiniLM-L6-v2/'
model = SentenceTransformer(model_path)




def start_system(session_name, topic_description ):
    clear_output(wait=False)
    
    if second_round:
        session_name=session_name+'_second_round'
    
    if os.path.exists(os.path.join(repository_path, f'sessions/scal/{session_name}')):
        scal = SCALRefugees.from_disk(session_name=session_name)
    else:    
        
        # LABELED & UNLABELED INFO.
        session_path = os.path.join(repository_path, f'sessions/scal/{session_name}/data/' ) 
        
        unlabeled = dataset.DatasetRefugees.get_unlabeled_items()
        item_representation_file=os.path.join(repository_path,'embeddings','item_representation_50k_final.pickle')
        with open(item_representation_file, 'rb') as reader:
            item_representation = pickle.load(reader)
            
        unlabeled = list(filter(lambda item: item.id_ in item_representation, unlabeled))[:20]
        weak_oracle = dataset.DatasetRefugees.get_weak_oracle()
        
        labeled = [dataset.QueryDatItemRefugees(topic_description)] #+ dataset.DatasetRefugees.get_initial_labeled_items()
        labeled[0].set_relevant()

#         labeled_ids=set([item.id_ for item in labeled])
#         unlabeled = list(filter(lambda item: not item.id_ in labeled_ids, unlabeled))
        scal = SCALRefugees(session_name, 
                            labeled, 
                            unlabeled,
                            batch_size_cap=cap,
                            weak_oracle=None,
                            topic_description=topic_description,
                            topic_vector=model.encode([topic_description])[0,:],
                            item_representation_file=item_representation_file,
                            seed=seed)
    scal.run()

_=SCAL_UI(start_system, second_round=second_round)